# Introduction to MCP (Model-Context Protocol)

https://github.com/alexeygrigorev/workshops/tree/main/agents-mcp

MCP (Model Context Protocol) is an open protocol developed by Anthropic that standardizes how AI models (or “agents”) connect and communicate with external tools, databases, and APIs.

They describe it as "USB Type-C for AI tools": it provides a universal interface that allows any AI agent to access a tool without needing custom integration code.

## How It Works

- MCP Server is a bridge between the agent and the tool.
- The agent communicates with the MCP server using a defined protocol.
- The MCP server handles logic for the tool (e.g., querying a database, searching documents, adding entries).
- The tool responds to the MCP server, which then passes the result back to the agent.

So developers only need to implement a tool once as an MCP server, and it becomes reusable by any agent that supports MCP.

## PostgreSQL example

Imagine you have a PostgreSQL database containing customer data, and you want your AI agent to query it.

Normally, every developer would have to write custom code for their agent to connect to PostgreSQL (handling authentication, queries, schemas, etc.).

With MCP, you can instead create a Postgres MCP Server:

1. The MCP server knows how to:
  - Connect to your PostgreSQL instance.
  - Retrieve database schemas.
  - Execute SQL queries, such as selecting customers from a specific city.

2. The AI agent doesn’t need to know SQL or PostgreSQL details — it just sends a standard MCP request like “Find all customers in London”.

3. The MCP server translates this into a SQL query, runs it on the PostgreSQL database, and returns a structured response with the matching records.

Now, any agent (running in Jupyter, VS Code, or another environment) can use this Postgres MCP server without writing any SQL or integration code.



## Creating MCP Server

Install:
```bash
pip install uv # if you don't have uv
uv init
uv add minsearch requests fastmcp
```

Create a file `search_tools.py`

In [1]:
from typing import List, Dict, Any

class SearchTools:

    def __init__(self, index):
        self.index = index

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the FAQ database for entries matching the given query.
    
        Args:
            query (str): Search query text to look up in the course FAQ.
    
        Returns:
            List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.
        """
        boost = {'question': 3.0, 'section': 0.5}
    
        results = self.index.search(
            query=query,
            filter_dict={'course': 'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5,
        )
    
        return results

    def add_entry(self, question: str, answer: str) -> None:
        """
        Add a new entry to the FAQ database.
    
        Args:
            question (str): The question to be added to the FAQ database.
            answer (str): The corresponding answer to the question.
        """
        doc = {
            'question': question,
            'text': answer,
            'section': 'user added',
            'course': 'data-engineering-zoomcamp'
        }
        self.index.append(doc)

Create a `main.py`

In [3]:
import requests 
from minsearch import AppendableIndex

from search_tools import SearchTools

def init_index():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    documents = []

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)


    index = AppendableIndex(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )

    index.fit(documents)
    return index


def init_tools():
    index = init_index()
    return SearchTools(index)


if __name__ == "__main__":
    tools = init_tools()
    print(tools.search("How do I install Kafka?"))

[{'text': 'confluent-kafka: `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`\nfastavro: pip install fastavro\nAbhirup Ghosh\nCan install Faust Library for Module 6 Python Version due to dependency conflicts?\nThe Faust repository and library is no longer maintained - https://github.com/robinhood/faust\nIf you do not know Java, you now have the option to follow the Python Videos 6.13 & 6.14 here https://www.youtube.com/watch?v=BgAlVknDFlQ&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=80  and follow the RedPanda Python version here https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/06-streaming/python/redpanda_example - NOTE: I highly recommend watching the Java videos to understand the concept of streaming but you can skip the coding parts - all will become clear when you get to the Python videos and RedPanda files.', 'section': 'Module 6: streaming with kafka', 'question': 'Python Kafka: Installing dependencies for python3 06-stream

Let's expose these tools with MCP. For that, we will need to create an MCP server. There are many frameworks for creating them. We will use [FastMCP](https://github.com/jlowin/fastmcp).

This is the simplest possible MCP server (taken from the docs):

In [ ]:
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()

Add this to `main.py`

In [ ]:
from fastmcp import FastMCP
from toyaikit.tools import wrap_instance_methods


def init_mcp():
    mcp = FastMCP("Demo 🚀")
    agent_tools = init_tools()
    wrap_instance_methods(mcp.tool, agent_tools)
    return mcp


if __name__ == "__main__":
    mcp = init_mcp()
    mcp.run()

Run it:

```bash
uv run python main.py
```

It uses standard input/output as transport:

```
 📦 Transport:       STDIO  
 ```

Which means, we can paste things into our terminal to test it (and simulate the interaction with the server)

# MCP STDIO Transport: Communicating with Local MCP Servers

## Handshake Sequence

First, we need to initialize the connection. We do it with the handshake sequence:

1. Send the ininialization request
2. Confirm the initialization
3. Now can see the list of available tools

Let's do this. Send the initialization request:

```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

We get back something like:

```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.13.1"}}}
```

This is a confirmation that we can proceed.

Next, confirm the initialization:

```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```

We don't get back anything

## Using Tools

Now we can see the list of available tools:

```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

We get back the list

```json
{"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"add_entry","description":"Add a new entry to the FAQ database.\n\nArgs:\n    question (str): The question to be added to the FAQ database.\n    answer (str): The corresponding answer to the question.","inputSchema":{"properties":{"question":{"type":"string"},"answer":{"type":"string"}},"required":["question","answer"],"type":"object"},"_meta":{"_fastmcp":{"tags":[]}}},{"name":"search","description":"Search the FAQ database for entries matching the given query.\n\nArgs:\n    query (str): Search query text to look up in the course FAQ.\n\nReturns:\n    List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.","inputSchema":{"properties":{"query":{"type":"string"}},"required":["query"],"type":"object"},"outputSchema":{"properties":{"result":{"items":{"additionalProperties":true,"type":"object"},"type":"array"}},"required":["result"],"type":"object","x-fastmcp-wrap-result":true},"_meta":{"_fastmcp":{"tags":[]}}}]}}
```

Formatted:

```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "tools": [
      {
        "name": "add_entry",
        "description": "Add a new entry to the FAQ database.\n\nArgs:\n    question (str): The question to be added to the FAQ database.\n    answer (str): The corresponding answer to the question.",
        "inputSchema": {
          "properties": {
            "question": {
              "type": "string"
            },
            "answer": {
              "type": "string"
            }
          },
          "required": [
            "question",
            "answer"
          ],
          "type": "object"
        },
        "_meta": {
          "_fastmcp": {
            "tags": [
              
            ]
          }
        }
      },
      {
        "name": "search",
        "description": "Search the FAQ database for entries matching the given query.\n\nArgs:\n    query (str): Search query text to look up in the course FAQ.\n\nReturns:\n    List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.",
        "inputSchema": {
          "properties": {
            "query": {
              "type": "string"
            }
          },
          "required": [
            "query"
          ],
          "type": "object"
        },
        "outputSchema": {
          "properties": {
            "result": {
              "items": {
                "additionalProperties": true,
                "type": "object"
              },
              "type": "array"
            }
          },
          "required": [
            "result"
          ],
          "type": "object",
          "x-fastmcp-wrap-result": true
        },
        "_meta": {
          "_fastmcp": {
            "tags": [
              
            ]
          }
        }
      }
    ]
  }
}
```

> Note: the schema for tools is somewhat similar to the OpenAI's function calling, but not the same.

Invoke a function:


```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "search", "arguments": {"query": "how do I run kafka?"}}}
```

And we get a response like:

```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"..."}],"isError":false}}
```


## Simple MCP Client (For Testing Only)

If you need an MCP client, you should use the built-in one from FastMCP. However, it's async (which is good for production cases), so testing it inside Jupyter is difficult.

Let's invoke it from Jupyter with ToyAIKit:



In [4]:
from toyaikit.mcp import MCPClient, SubprocessMCPTransport

command = "uv run python main.py".split()
workdir = "mcp"

client = MCPClient(
    transport=SubprocessMCPTransport(
        server_command=command,
        workdir=workdir
    )
)

In [ ]:
# We can:

# Start the server (run the command)
client.start_server()

# Send "initialize"
client.initialize()

# Send "initialized"
client.initialized()

# get tools
client.get_tools()

Started server with command: uv run python main.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.16.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['add_entry', 'search']


[{'name': 'add_entry',
  'description': 'Add a new entry to the FAQ database.\n\nArgs:\n    question (str): The question to be added to the FAQ database.\n    answer (str): The corresponding answer to the question.',
  'inputSchema': {'properties': {'question': {'type': 'string'},
    'answer': {'type': 'string'}},
   'required': ['question', 'answer'],
   'type': 'object'},
  '_meta': {'_fastmcp': {'tags': []}}},
 {'name': 'search',
  'description': 'Search the FAQ database for entries matching the given query.\n\nArgs:\n    query (str): Search query text to look up in the course FAQ.\n\nReturns:\n    List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.',
  'inputSchema': {'properties': {'query': {'type': 'string'}},
   'required': ['query'],
   'type': 'object'},
  'outputSchema': {'properties': {'result': {'items': {'additionalProperties': True,
      'type': 'object'},
     'type': 'array'}},
   'required': ['result'],
   'type': 'object',
   'x

We can do all 4 with one command:

In [6]:
client.full_initialize()

Started server with command: uv run python main.py
Waiting 0.5 seconds for server to stabilize...
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.16.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['add_entry', 'search']


Lets invoke the search tool:

In [9]:
result = client.call_tool('search', {'query': 'how do I run docker?'})

Calling tool 'search' with arguments: {'query': 'how do I run docker?'}


In [10]:
print(result)

{'content': [{'type': 'text', 'text': '[{"text":"When you run this command second time\\ndocker run -it \\\\\\n-e POSTGRES_USER=\\"root\\" \\\\\\n-e POSTGRES_PASSWORD=\\"root\\" \\\\\\n-e POSTGRES_DB=\\"ny_taxi\\" \\\\\\n-v <your path>:/var/lib/postgresql/data \\\\\\n-p 5432:5432 \\\\\\npostgres:13\\nThe error message above could happen. That means you should not mount on the second run. This command helped me:\\nWhen you run this command second time\\ndocker run -it \\\\\\n-e POSTGRES_USER=\\"root\\" \\\\\\n-e POSTGRES_PASSWORD=\\"root\\" \\\\\\n-e POSTGRES_DB=\\"ny_taxi\\" \\\\\\n-p 5432:5432 \\\\\\npostgres:13","section":"Module 1: Docker and Terraform","question":"Docker - Error response from daemon: error while creating buildmount source path \'/run/desktop/mnt/host/c/<your path>\': mkdir /run/desktop/mnt/host/c: file exists","course":"data-engineering-zoomcamp"},{"text":"You may have this error:\\n$ docker run -it ubuntu bash\\nthe input device is not a TTY. If you are using mint

If we want to use MCP with plan OpenAI API, we need to convert the MCP tools into the calling schemas. We can do it with ToyAIKit, you can see the code [here](https://github.com/alexeygrigorev/toyaikit/blob/main/toyaikit/mcp/mcp_tools.py#L4)

In [1]:
from toyaikit.mcp import MCPClient, SubprocessMCPTransport

command = "uv run python main.py".split()
workdir = "mcp"

client = MCPClient(
    transport=SubprocessMCPTransport(
        server_command=command,
        workdir=workdir
    )
)

In [6]:
client.full_initialize()

Started server with command: uv run python main.py
Waiting 0.5 seconds for server to stabilize...
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.16.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['add_entry', 'search']


In [7]:
developer_prompt = """
You are an assistant answer question from the user.
"""

In [8]:
from toyaikit.llm import OpenAIClient
from toyaikit.mcp import MCPTools
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner

mcp_tools = MCPTools(client)

chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    llm_client=OpenAIClient(model='gpt-4o-mini')
)

Run it:

In [ ]:
runner.run();

You: How do I install kafka?


Retrieving available tools...
Available tools: ['add_entry', 'search']


If we want to use the client from FastMCP, it looks like that (but we won't do it here):

In [ ]:
import asyncio

async def main():
    async with Client("uv run python main.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())